In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [2]:
TEST_SIZE = 7
DATSETS = [
    '/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/covid/casos_confirmados.csv',
    '/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/sinteticos/sinteticos.csv',
    '/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/temperatura_minima_diaria/temperaturas.csv'
]
ORDER = (1, 1, 1)

In [5]:
ts = pd.read_csv('/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/covid/casos_confirmados.csv', index_col=0, sep=';')
train_ts = ts.iloc[:-TEST_SIZE].values
%timeit ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
model = ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
%timeit forecast = model.forecast(steps=TEST_SIZE)

145 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.69 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
ts = pd.read_csv('/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/sinteticos/sinteticos.csv', index_col=0, sep=';')
train_ts = ts.iloc[:-TEST_SIZE].values
%timeit ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
model = ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
%timeit forecast = model.forecast(steps=TEST_SIZE)

86.3 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
3.13 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
ts = pd.read_csv('/home/jvdavim/git/TCC-Trabalho-Conclusao-Curso/data/temperatura_minima_diaria/temperaturas.csv', index_col=0, sep=';')
train_ts = ts.iloc[:-TEST_SIZE].values
%timeit ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
model = ARIMA(train_ts, order=ORDER, enforce_stationarity=False, enforce_invertibility=False).fit()
%timeit forecast = model.forecast(steps=TEST_SIZE)

579 ms ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.65 ms ± 607 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
